In [1]:
file1= r'D:\Naresh\GUVI\Projects\Microsoft\raw_data\GUIDE_Train.csv'
file2 = r'D:\Naresh\GUVI\Projects\Microsoft\raw_data\GUIDE_Test.csv'
file3=r'D:\Naresh\GUVI\Projects\Microsoft\raw_data\new_train_sample.csv'

In [ ]:
import pandas as pd

# train = pd.read_csv(file1)
train_2=pd.read_csv(file3)
test = pd.read_csv(file2)

In [ ]:
# print(f'Lenght of train : {len(train)}')
print(f'Lenght of train 2 : {len(train_2)}')
print(f'Lenght of test : {len(test)}')

Lenght of train 2 : 4758418


In [4]:
train_df=pd.DataFrame
train_2_df =pd.DataFrame
test_df=pd.DataFrame

In [ ]:
# train.drop(columns=['Timestamp'], inplace=True)
train_2.drop(columns=['Timestamp'], inplace=True)
test.drop(columns=['Timestamp'], inplace=True)

In [6]:

tra = train_2
nan_count = tra.isna().sum()
nan_percentage = (nan_count / len(tra)) * 100
duplicates_in_columns = tra.apply(lambda col: col.duplicated(keep=False).any())
# variance_data = tra.var()
summary = pd.DataFrame({
    'NaN Count': nan_count,
    'NaN Percentage': nan_percentage,
    'Duplicates' : duplicates_in_columns,
    # 'variance_data' :variance_data
})

train_2_df = summary


In [7]:
output_path="D:/Naresh/GUVI/Projects/Microsoft/data/processed_data/"

with pd.ExcelWriter(output_path+'analysis_data.xlsx') as writer:
        print("writing data in excel...")
        # train_df.to_excel(writer, sheet_name='Train 1',index=True)
        train_2_df.to_excel(writer, sheet_name='Train 2',index=True)
        # test_df.to_excel(writer, sheet_name='Test',index=True)

writing data in excel...


In [8]:
columns_to_keep = nan_percentage[nan_percentage < 50].index

In [ ]:
df_filtered=train_2[columns_to_keep]
df_filtered = df_filtered[df_filtered['IncidentGrade'].notna()]

df_test = test[columns_to_keep]
df_test=df_test[df_test['IncidentGrade'].notna()]

In [37]:
categorical_features=[]
numerical_features =[]
for i in df_filtered.columns:

    if type(df_filtered[i][0])== str:
        categorical_features.append(i)
    else:
        numerical_features.append(i)

In [ ]:

from scipy import stats
from sklearn.preprocessing import LabelEncoder


#Label encode the IncidentGrade (if necessary) since it's categorical
target_encoder = LabelEncoder()
# df_filtered['target_encoded'] = target_encoder.fit_transform(df_filtered['IncidentGrade'])

#  ANOVA for numerical features with categorical IncidentGrade
anova_results = {}

for feature in numerical_features:
    # Perform ANOVA for each numerical feature against the IncidentGrade
    group_data = [df_filtered[feature][df_filtered['IncidentGrade'] == category] for category in df_filtered['IncidentGrade'].unique()]
    f_stat, p_value = stats.f_oneway(*group_data)
    anova_results[feature] = (f_stat, p_value)

print("ANOVA Results for Numerical Features:")
for feature, (f_stat, p_value) in anova_results.items():
    print(f"{feature}: F-statistic = {f_stat}, p-value = {p_value}")

# Chi-Square for categorical features with categorical IncidentGrade

chi2_results = {}

for feature in categorical_features:
    # Create contingency table for each categorical feature
    contingency_table = pd.crosstab(df_filtered[feature], df_filtered['IncidentGrade'])
    
    # Perform Chi-Square test
    chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)
    chi2_results[feature] = (chi2_stat, p_value)

print("\nChi-Square Results for Categorical Features:")
for feature, (chi2_stat, p_value) in chi2_results.items():
    print(f"{feature}: Chi2-statistic = {chi2_stat}, p-value = {p_value}")


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


ANOVA Results for Numerical Features:
Unnamed: 0: F-statistic = 0.22925829642951948, p-value = 0.7951231382876303
Id: F-statistic = 1439.484198513051, p-value = 0.0
OrgId: F-statistic = 58997.76021886329, p-value = 0.0
IncidentId: F-statistic = 76452.55955725766, p-value = 0.0
AlertId: F-statistic = 47081.29268933218, p-value = 0.0
DetectorId: F-statistic = 32908.98954426741, p-value = 0.0
AlertTitle: F-statistic = 48290.78906666099, p-value = 0.0
DeviceId: F-statistic = 15635.311122018205, p-value = 0.0
Sha256: F-statistic = 47594.834053318205, p-value = 0.0
IpAddress: F-statistic = 58524.64720546779, p-value = 0.0
Url: F-statistic = 21112.06672260003, p-value = 0.0
AccountSid: F-statistic = 24636.7169202503, p-value = 0.0
AccountUpn: F-statistic = 4759.04937297763, p-value = 0.0
AccountObjectId: F-statistic = 26743.9579418214, p-value = 0.0
AccountName: F-statistic = 17730.550232143647, p-value = 0.0
DeviceName: F-statistic = 50484.52016148558, p-value = 0.0
NetworkMessageId: F-stati

In [50]:
columns_to_remove=[]
for values in anova_results:
    if anova_results[values][1] != 0.0:
        columns_to_remove.append(values)
for values in chi2_results:
    if chi2_results[values][1] != 0.0:
        columns_to_remove.append(values)


In [55]:
print(categorical_features)

['Category', 'IncidentGrade', 'EntityType', 'EvidenceRole']


In [52]:
df_filtered.drop(columns=columns_to_remove, inplace=True)

In [57]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for i in categorical_features:
    df_filtered[i]=le.fit_transform(df_filtered[i])


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Example data: X (features) and y (target)
y = df_filtered['IncidentGrade']
X = df_filtered.drop('IncidentGrade', axis=1)  # All features

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit a Random Forest Classifier to calculate feature importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importances
feature_importances = rf.feature_importances_

# Create a DataFrame to view the importance of each feature
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print(feature_importance_df)


In [ ]:
accuracy = rf.score(X_test, y_test)
print(f"Model Accuracy: {accuracy:.2f}")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

selected_features = ['OrgId', 'DetectorId','AlertTitle','AlertId','Category','CountryCode','EntityType']  # Example
X_selected = X[selected_features]

# Fit a Random Forest Classifier and evaluate using cross-validation
rf = RandomForestClassifier(n_estimators=100, random_state=42)
scores = cross_val_score(rf, X_selected, y, cv=5)

print(f"Cross-validated scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
